<a id='top'></a>
# NIRSpec MOS pipeline processing

**Author**: James Muzerolle

Plotting function originally developed by Bryan Hilbert

**Latest Update**: 15 October 2021
test

## Table of Contents
* [Introduction](#intro)
    * [Overview](#overview)
    * [Simulated data](#sims)
* [Imports](#imports)
* [Convenience functions](#func)
* [Pipeline processing flag](#flag)
* [Input simulations](#inputs)
* [Association files and metadata](#associations)
* [Run the calwebb_spec2 pipeline](#runspec2)
* [Run the calwebb_spec3 pipeline](#runspec3)
* [Master background subtraction](#mbs)

## Introduction <a id='intro'></a>

In this notebook, we will explore the stage 2 and 3 pipelines for NIRSpec MOS data. Here, we will focus on the mechanics of processing "real" example data, including how to use associations for exposure specification and multi-exposure combination, the role of metadata, and what the primary data products at each stage look like. We will also see examples of how to interact and work with data models and metadata.

We are using pipeline version 1.2.3 for all data processing in this notebook. Most of the processing runs shown here use the default reference files from the Calibration Reference Data System (CRDS). Please note that pipeline software development is a continuous process, so results in some cases may be slightly different if using a different version. There are also a few known issues with some of the pipeline steps in this build that are expected to be fixed in the near future, though these do not significantly effect the products you will see here. Finally, some of the calibration reference files used by individual pipeline steps in the current CRDS context are placeholders, as they require calibrations that can only be taken in flight; this means that the absolute flux values and formal uncertainties in the products shown here are not physical and should not be taken literally.

### Simulated data <a id='sims'></a>

We will be using simulated NIRSpec exposures generated by the ESA Instrument Performance Simulator (IPS). The simulations were created using an input scene consisting of 60 point sources within the MSA field of view, each described by an emission-line galaxy template spectrum at different redshifts. The observation included a series of three nodded exposures, in which each source is moved to a different shutter within its three-shutter slitlet. The instrument was configured with two different disperser settings: G235M+F170LP and G395M+F290LP. The nodded exposures are numbered for convenience in these example file names according to the ordering of the positions in the pattern. These file names are not indicative of actual data product file names you will see in the archive.

There are a number of caveats to be aware of regarding these simulated data. 1) The IPS does not include a full treatment of all of the effects corrected by the stage 2 pipeline, particularly some of the throughput components. As with the above caveat regarding reference files, the simulations shown here should not be used for any analyses of flux information. 2) The simulated PSF is truncated in order to save on computation time, which results in an artifical drop in apparent count rate in the PSF wings in some cases. 3) Some of the error arrays in the rate files are not populated, as those are normally generated during stage 1 of the pipeline. In addition to the reference file issue noted above, this means the final combined error values should be ignored. 4) Spacecraft pointing-related information has to be added by-hand to the headers before the simulated data can be processed by the pipeline. These keywords are used by the stage 3 pipeline when combining exposures, in order to align the spectral traces. In addition, the MSA metafile, used by the pipeline to determine open shutter and source information, also had to be created by-hand.  Because this has to be a manual process and may be subject to small errors, the quality of the combined products here should not be taken as indicative of actual in-flight performance.

***Note that these simulations are preliminary and will be replaced with newer versions in the near future, though after the webinar takes place.  For this reason, these data should only be used in the context of running this notebook, and we request that you not share the data.***

## Imports <a id='imports'></a>

Import packages necessary for this notebook

In [ ]:
import numpy as np

import glob

import os
import zipfile
import urllib.request

import json

from shutil import copyfile

from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

plt.rcParams.update({'font.size': 18})

Import JWST pipeline modules

In [ ]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# individual steps
from jwst.assign_wcs import AssignWcsStep
from jwst.assign_wcs import nirspec
from jwst.background import BackgroundStep
from jwst.imprint import ImprintStep
from jwst.msaflagopen import MSAFlagOpenStep
from jwst.extract_2d import Extract2dStep
from jwst.srctype import SourceTypeStep
from jwst.wavecorr import WavecorrStep
from jwst.flatfield import FlatFieldStep
from jwst.pathloss import PathLossStep
from jwst.photom import PhotomStep
from jwst.cube_build import CubeBuildStep
from jwst.extract_1d import Extract1dStep
from jwst.combine_1d import Combine1dStep

# data models
from jwst import datamodels

# associations
from jwst.associations import asn_from_list

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> Begin PEP8 check cells (delete below when finished)</b></p>

In [ ]:
# disable all imported packages' loggers
import logging
logging.root.manager.loggerDict = {}

In [ ]:
# enable PEP8 checker for this notebook
%load_ext pycodestyle_magic
%flake8_on --ignore E261,E501,W291,W293

# only allow the checker to throw warnings when there's a violation
logging.getLogger('flake8').setLevel('ERROR')
logging.getLogger('stpipe').setLevel('ERROR')

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> End PEP8 check cells (delete above when finished)</b></p>

## Define convenience functions and parameters <a id='func'></a>

In [ ]:
# All files created by the steps in this notebook have been pre-computed and cached on Box

# first specify a desired local directory in which to place the downloaded data, as well as any offline processing you choose to run
output_dir = './nirspec_files/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# set the Box link and file name
ziplink = 'https://stsci.box.com/shared/static/8iwpmppwdp9pyaun2axk7z0831esc3ox.zip'
zipfilename = 'nirspec_data.zip'
if not os.path.isfile(os.path.join(output_dir, zipfilename)):
    print('Downloading {}...'.format(zipfilename))
    demo_file = download_file(ziplink, cache=True)
    # Make a symbolic link using a local name for convenience
    os.symlink(demo_file, os.path.join(output_dir, zipfilename))
else:
    print('{} already exists, skipping download...'.format(zipfilename))

# unzip
zf = zipfile.ZipFile(output_dir+'nirspec_data.zip', 'r')
zf.extractall(output_dir)

In [ ]:
def show_image(data_2d, vmin, vmax, xsize=19, ysize=10, title=None, aspect=1, scale='log', units='MJy/sr'):
    """Function to generate a 2D, log-scaled image of the data
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    elif scale == 'Asinh':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=AsinhStretch())
    fig = plt.figure(figsize=(xsize, ysize))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm, aspect=aspect, cmap='gist_earth')

    if (units != 'none'):
        fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

## Pipeline processing flag <a id='flag'></a>

The pipeline and individual steps take too long to run in real time for this demo, so all products shown here have been pre-computed, and the actual pipeline calls will be skipped. Change the following flag to True if you want to run everything offline.

In [ ]:
runflag = False

## Input simulations <a id='inputs'></a>

We will be using simulated NIRSpec exposures of point sources observed with the MSA. Because the simulator generates count rate maps, equivalent to level 2a data products, we have to skip stage 1 of the pipeline and instead start the processing with the calwebb_spec2 pipeline. First, let's take a look at a few of the level 2a images to get familiarized with the inputs. The observation consists of 3 nodded exposures, with each source moving to a different shutter in its 3-shutter slitlet.

In [ ]:
# get the data model of dither position 1:
ratefile1 = output_dir+'nirspec_mossim_d1_g235m_nrs1_rate.fits' # for the NRS1 detector
dither1 = datamodels.open(ratefile1)
ratefile2 = output_dir+'nirspec_mossim_d1_g235m_nrs2_rate.fits' # for the NRS2 detector
dither2 = datamodels.open(ratefile2)

# get the pixel data (the SCI extension of the fits file)
ratesci1 = dither1.data
ratesci2 = dither2.data

# display the images
show_image(ratesci1, 0, 1., xsize=19, ysize=19, units='DN/s', title='NRS1')
show_image(ratesci2, 0, 1., xsize=19, ysize=19, units='DN/s', title='NRS2')

# zoom in to show details of one slitlet
show_image(ratesci1[1550:1650,850:1050], 0, 0.1, xsize=19, ysize=15, units='none', scale='Asinh', title='zoom of NRS1')

2021-10-18 15:38:10,168 - stpipe - INFO - 16:30: E231 missing whitespace after ','


## Association files and metadata <a id='associations'></a>

One purpose of the nodded exposures in this case is to enable background subtraction.  The spec2 pipeline is set up to handle this by using association files that list the exposures to be used as backgrounds for each input exposure.  The background elements can also be specified manually as inputs into the "background" step, but we'll be using association files in this example.

In [ ]:
# show the contents of one of the association files
asn_file = output_dir+"l2_g235m_asn.json"
with open(asn_file) as f_obj:
  asn_data = json.load(f_obj)
asn_data

2021-10-18 15:38:14,640 - stpipe - INFO - 4:3: E111 indentation is not a multiple of four


One unique aspect of MOS data processing is that it requires additional metadata, such as shutter locations and source positions. The pipeline gets this information from an MSA "metafile" that is automatically generated, along with the raw "level 1b" files, using metadata taken from the PPSDB. The metadata is originally populated when the observation is created with the MSA planning tool in APT. The metafile is a fits file containing two binary fits tables, one with MSA shutter information for each slitlet and nod, and the other with various details about each source observed.  Let's take a look at the metafile for this simulation:

In [ ]:
# open the MSA metafile
metafile = output_dir+'pointing_summary_n_metafile_msa.fits'
hdul = fits.open(metafile)
hdul.info()

In [ ]:
# columns of the "SHUTTER INFO" table
hdul['SHUTTER_INFO'].columns

In the "SHUTTER_INFO" table, the number of rows per slitlet is N_s x N_n, where N_s is the number of open shutters in each slitlet and N_n is the number of nods. In this case, N_s = 3 and N_n = 3, so there are 9 rows per slitlet. These are needed by the pipeline to determine which slitlet shutter contains the source in each nodded exposure.

In [ ]:
# print table entries for the first two slitlets
print(hdul['SHUTTER_INFO'].data[:18])

In [ ]:
# columns of the "SOURCE_INFO" table
hdul['SOURCE_INFO'].columns

The SOURCE_INFO table contains one row per source that was observed with an MSA configuration. The most important information here are the source RA & Dec coordinates, and the stellarity parameter, which is used to determine whether the source should be processed as point or extended.

In [ ]:
# print table entries for the first 10 sources
print(hdul['SOURCE_INFO'].data[:10])

## Run the spec2 pipeline <a id='runspec2'></a>

In [ ]:
# run the calwebb_spec2 pipeline using an association file as input

if (runflag == True):
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    result = spec2(asn_file)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# take a look at the results - open the level 2b files

callist = [f for f in glob.glob(output_dir+"*g235m_nrs?_cal.fits")]
callist.sort()
for calfile in callist:
    print(calfile)
    cal = datamodels.open(calfile) # this contains the calibrated unrectified 2D spectra
    root = calfile[:-9]
    s2d = datamodels.open(root+'_s2d.fits')  # this contains the calibrated *rectified* 2D spectra
    x1d = datamodels.open(root+'_x1d.fits')  # this contains the aperture-extracted 1D spectra
    
    for i, slit in enumerate(cal.slits):

        if (slit.name == '31'):  # change this, or comment out, to see other slits
            print(slit.name)
        
            calsci = slit.data  # contains the pixel data from the cal file (SCI extension)
            s2dsci = s2d.slits[i].data  # contains the pixel data from the s2d file
    
            # determine the wavelength scale of the s2d data for plotting purposes
            # get the data model WCS object
            wcsobj = s2d.slits[i].meta.wcs
            y, x = np.mgrid[:s2dsci.shape[0], : s2dsci.shape[1]]  # grid of pixel x,y indices
            det2sky = wcsobj.get_transform('detector','world')  # the coordinate transform from detector space (pixels) to sky (RA, DEC in degrees)
            ra, dec, s2dwave = det2sky(x, y)  # RA, Dec, wavelength (microns) for each pixel
            s2dwaves = s2dwave[0, :]  # only need a single row of values since this is the rectified spectrum
            xtint = np.arange(100, s2dsci.shape[1], 100)
            xtlab = np.round(s2dwaves[xtint], 2)  # wavelength labels for the x-axis
        
            # get wavelength & flux from the x1d data model
            x1dwave = x1d.spec[i].spec_table.WAVELENGTH
            x1dflux = x1d.spec[i].spec_table.FLUX
  
            # plot the unrectified calibrated 2D spectrum
            show_image(calsci, -0.01, 0.01, aspect=5., scale='linear', units='MJy')
        
            # plot the rectified 2D spectrum
            show_image(s2dsci, -0.01, 0.01, aspect=5., scale='linear', units='MJy')
            plt.xticks(xtint, xtlab)
            plt.xlabel('wavelength (microns)')
        
            # plot the 1D extracted spectrum
            fig = plt.figure(figsize=(19,8))
            plt.plot(x1dwave, x1dflux)
            plt.xlabel('wavelength (microns)')
            plt.ylabel('flux (Jy)')
            plt.show()

2021-10-18 15:39:11,242 - stpipe - INFO - 24:54: E231 missing whitespace after ','
2021-10-18 15:39:11,244 - stpipe - INFO - 43:41: E231 missing whitespace after ','


ValueError: Column names don't match schema. Schema has {'berror', 'error'}. Data has {'sb_var_flat', 'flux_var_rnoise', 'flux_var_poisson', 'flux_var_flat', 'bkgd_var_poisson', 'flux_error', 'bkgd_var_rnoise', 'bkgd_error', 'bkgd_var_flat', 'sb_var_poisson', 'sb_var_rnoise'}

Now we'll process the G395M data, using the same methodology (to be used later).

In [ ]:
# association file
asn_file = output_dir+"l2_g395m_asn.json"

In [ ]:
# run the calwebb_spec2 pipeline using association files as inputs

if (runflag == True):
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    result = spec2(asn_file)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

## Run the calwebb_spec3 pipeline <a id='runspec3'></a>

For an observation that contains multiple exposures of the same sources, such as the nod set in this example, this pipeline will combine all of the exposures into a single output product.  Both a rectified 2D and extracted 1D spectrum are generated.  This process also includes an outlier detection step that compares the stack of values at each resampled pixel and flags outliers based on noise threshold parameters; the flagged outliers are not included in the spectral combination.  However, we will skip that step in this example because of the aforementioned limitations with the noise propagation, as well as the fact that the simulations do not include outliers.

In [ ]:
# create an association file with all the level 2b cal.fits products generated above

if (runflag == True):
    outfilename = 'nirspec_mossim_g235m_{source_id}_combined.fits'
    calfilelist = [os.path.basename(f) for f in glob.glob(output_dir+'*g235m*_cal.fits')]
    from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
    asn = asn_from_list.asn_from_list(calfilelist, product_name=outfilename)

    asn_file_l3 = output_dir+'l3_g235m_asn.json'
    with open(asn_file_l3, 'w') as outfile:
        name, serialized = asn.dump(format='json')
        outfile.write(serialized)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# run the calwebb_spec3 pipeline using the association file as input

if (runflag == True):
    spec3 = Spec3Pipeline()
    spec3.save_results = True
    spec3.output_dir = output_dir
    spec3.outlier_detection.skip = True
    result = spec3(asn_file_l3)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# display the level 3 combined products

s2d3list = [f for f in sorted(glob.glob(output_dir+'*g235m*_combined_s2d.fits'))]
x1d3list = [f for f in sorted(glob.glob(output_dir+'*g235m*_combined_x1d.fits'))]

for i, file in enumerate(s2d3list):
    s2d3 = datamodels.open(file)
    
    if (s2d3.name == '31'):  # show just one of the files for brevity's sake; change or comment out to see others
        print(file)
        print(s2d3.name)
        s2d3sci = s2d3.data
        print(x1d3list[i])
        x1d3 = datamodels.open(x1d3list[i])
        x1dwave = x1d3.spec[0].spec_table.WAVELENGTH
        x1dflux = x1d3.spec[0].spec_table.FLUX

        # get the wavelength scale
        wcsobj = s2d3.meta.wcs
        y, x = np.mgrid[:s2d3sci.shape[0], : s2d3sci.shape[1]]
        det2sky = wcsobj.get_transform('detector', 'world')
        s2d3ra, s2d3dec, s2d3wave = det2sky(x, y)
        s2d3waves = s2d3wave[0, :]
        xtint = np.arange(100, s2d3sci.shape[1], 100)
        xtlab = np.round(s2d3waves[xtint], 2)

        # show the combined s2d image
        show_image(s2d3sci, -0.01, 0.01, aspect=5., scale='linear', units='MJy')
        plt.xticks(xtint, xtlab)
        plt.xlabel('wavelength (microns)')

        # plot extracted 1D spectrum
        fig = plt.figure(figsize=(19,8))
        plt.plot(x1dwave,x1dflux)
        plt.xlabel('wavelength (microns)')
        plt.ylabel('flux (Jy)')
        plt.ylim(-5.e4,6.e5)
        plt.show()

2021-10-18 16:58:41,186 - stpipe - INFO - 33:37: E231 missing whitespace after ','
2021-10-18 16:58:41,187 - stpipe - INFO - 34:25: E231 missing whitespace after ','
2021-10-18 16:58:41,188 - stpipe - INFO - 37:23: E231 missing whitespace after ','


ValueError: Column names don't match schema. Schema has {'berror', 'error'}. Data has {'sb_var_flat', 'flux_var_rnoise', 'flux_var_poisson', 'flux_var_flat', 'bkgd_var_poisson', 'flux_error', 'bkgd_var_rnoise', 'bkgd_error', 'bkgd_var_flat', 'sb_var_poisson', 'sb_var_rnoise'}

Now let's combine the G395M data.

In [ ]:
# create an association file with all the level 2b cal.fits products generated above

if (runflag == True):
    outfilename = 'nirspec_mossim_g395m_{source_id}_combined.fits'
    calfilelist = [os.path.basename(f) for f in glob.glob(output_dir+'*g395m*_cal.fits')]
    from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
    asn = asn_from_list.asn_from_list(calfilelist, product_name=outfilename)

    asn_file_l3 = output_dir+'l3_g395m_asn.json'
    with open(asn_file_l3, 'w') as outfile:
        name, serialized = asn.dump(format='json')
        outfile.write(serialized)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# run the calwebb_spec3 pipeline using the association file as input

if (runflag == True):
    spec3 = Spec3Pipeline()
    spec3.save_results = True
    spec3.output_dir = output_dir
    spec3.outlier_detection.skip = True  # skip this step for now, because the simulations do not include noise
    result = spec3(asn_file_l3)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# display the level 3 combined products

s2d3list = [f for f in sorted(glob.glob(output_dir+'*g395m*_combined_s2d.fits'))]
x1d3list = [f for f in sorted(glob.glob(output_dir+'*g395m*_combined_x1d.fits'))]

for i, file in enumerate(s2d3list):
    s2d3 = datamodels.open(file)
    
    if (s2d3.name == '31'):  # show just one of the files for brevity's sake; change or comment out to see others
        print(file)
        print(s2d3.name)
        s2d3sci = s2d3.data
        print(x1d3list[i])
        x1d3 = datamodels.open(x1d3list[i])
        x1dwave = x1d3.spec[0].spec_table.WAVELENGTH
        x1dflux = x1d3.spec[0].spec_table.FLUX

        # get the wavelength scale
        wcsobj = s2d3.meta.wcs
        y, x = np.mgrid[:s2d3sci.shape[0], : s2d3sci.shape[1]]
        det2sky = wcsobj.get_transform('detector', 'world')
        s2d3ra, s2d3dec, s2d3wave = det2sky(x, y)
        s2d3waves = s2d3wave[0, :]
        xtint = np.arange(100, s2d3sci.shape[1], 100)
        xtlab = np.round(s2d3waves[xtint], 2)

        # show the combined s2d image       
        show_image(s2d3sci, -0.01, 0.01, aspect=5., scale='linear', units='MJy')
        plt.xticks(xtint, xtlab)
        plt.xlabel('wavelength (microns)')

        # plot extracted 1D spectrum
        fig = plt.figure(figsize=(19,8))
        plt.plot(x1dwave,x1dflux)
        plt.xlabel('wavelength (microns)')
        plt.ylabel('flux (Jy)')
        plt.show()

2021-10-18 16:59:37,701 - stpipe - INFO - 33:37: E231 missing whitespace after ','
2021-10-18 16:59:37,703 - stpipe - INFO - 34:25: E231 missing whitespace after ','


ValueError: Column names don't match schema. Schema has {'berror', 'error'}. Data has {'sb_var_flat', 'flux_var_rnoise', 'flux_var_poisson', 'flux_var_flat', 'bkgd_var_poisson', 'flux_error', 'bkgd_var_rnoise', 'bkgd_error', 'bkgd_var_flat', 'sb_var_poisson', 'sb_var_rnoise'}

Now that we have level 3 spectra for both gratings, these can be combined at the 1D level.

In [ ]:
# association file with 1D spectra per source per grating
asn_file = output_dir+'nirspec_mossim_all_s02140.json'

if (runflag == True):
    step = Combine1dStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(asn_file)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
x1dcomb = datamodels.open(output_dir+'nirspec_mossim_all_s02140_combine1dstep.fits')
x1dcombwave = x1dcomb.spec[0].spec_table.WAVELENGTH
x1dcombflux = x1dcomb.spec[0].spec_table.FLUX

# plot combined 1D spectrum
fig = plt.figure(figsize=(19, 8))
plt.plot(x1dcombwave, x1dcombflux)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.ylim(-5.e4,2.e5)
plt.show()

2021-10-18 16:59:52,972 - stpipe - INFO - 10:15: E231 missing whitespace after ','


## Master background subtraction using MSA background shutters <a id='mbs'></a>

We'll now explore one example of the master background subraction methodology for MOS data.  There are multiple ways of creating a master background; the most common will likely be programs that have designed specific background shutters for that purpose.  The simulations we are using here do not include such dedicated backgrounds, but some of the targets have minimal S/N and hence are dominated by the background emission, so we will use those as a proxy after some minipulation of the MSA metafile.

For comparison purposes, we will first reprocess one of the exposures without doing any background subtraction.
To simplify for this demonstration, we will use just one of the sources observed in the exposure (source ID 2140, slitlet ID 31), isolating it by editing the MSA metafile appropriately to include only one row in the SHUTTER_INFO table.

In [ ]:
# edit the metafile to include only a single row corresponding to the source and exposure we want to use

if (runflag == True):

    hdul = fits.open(metafile)

    # pick the row in the original metafile SHUTTER_INFO table with a reasonably bright source in the appropriate shutter for this particular nod position
    indices = [255]

    # find the matching values for each of the columns
    slitlets = hdul['SHUTTER_INFO'].data['SLITLET_ID'][indices]
    metaids = hdul['SHUTTER_INFO'].data['MSA_METADATA_ID'][indices]
    quads = hdul['SHUTTER_INFO'].data['SHUTTER_QUADRANT'][indices]
    rows = hdul['SHUTTER_INFO'].data['SHUTTER_ROW'][indices]
    cols = hdul['SHUTTER_INFO'].data['SHUTTER_COLUMN'][indices]
    sources = hdul['SHUTTER_INFO'].data['SOURCE_ID'][indices]
    bkgd = hdul['SHUTTER_INFO'].data['BACKGROUND'][indices]
    state = hdul['SHUTTER_INFO'].data['SHUTTER_STATE'][indices]
    srcx = hdul['SHUTTER_INFO'].data['ESTIMATED_SOURCE_IN_SHUTTER_X'][indices]
    srcy = hdul['SHUTTER_INFO'].data['ESTIMATED_SOURCE_IN_SHUTTER_Y'][indices]
    dith = hdul['SHUTTER_INFO'].data['DITHER_POINT_INDEX'][indices]
    primary = hdul['SHUTTER_INFO'].data['PRIMARY_SOURCE'][indices]

    print(slitlets)
    print(bkgd)
    print(primary)

    # construct a new table with just these values
    tabcol1 = fits.Column(name='SLITLET_ID', format='I', array=slitlets)
    tabcol2 = fits.Column(name='MSA_METADATA_ID', format='I', array=metaids)
    tabcol3 = fits.Column(name='SHUTTER_QUADRANT', format='I', array=quads)
    tabcol4 = fits.Column(name='SHUTTER_ROW', format='I', array=rows)
    tabcol5 = fits.Column(name='SHUTTER_COLUMN', format='I', array=cols)
    tabcol6 = fits.Column(name='SOURCE_ID', format='I', array=sources)
    tabcol7 = fits.Column(name='BACKGROUND', format='A', array=bkgd)
    tabcol8 = fits.Column(name='SHUTTER_STATE', format='4A', array=state)
    tabcol9 = fits.Column(name='ESTIMATED_SOURCE_IN_SHUTTER_X', format='E' ,array=srcx)
    tabcol10 = fits.Column(name='ESTIMATED_SOURCE_IN_SHUTTER_Y', format='E', array=srcy)
    tabcol11 = fits.Column(name='DITHER_POINT_INDEX', format='I', array=dith)
    tabcol12 = fits.Column(name='PRIMARY_SOURCE', format='1A', array=primary)
    hdul['SHUTTER_INFO'] = fits.BinTableHDU.from_columns([tabcol1, tabcol2, tabcol3, tabcol4, tabcol5,
                                                          tabcol6, tabcol7, tabcol8, tabcol9, tabcol10,
                                                          tabcol11, tabcol12], name='SHUTTER_INFO')

    # save to a new metafile
    hdul.writeto(output_dir+'newmetafile_1shutter_msa.fits')
    hdul.close()

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# take a look at the contents of this new metafile

metafile = output_dir+'newmetafile_1shutter_msa.fits'
hdul = fits.open(metafile)

print(hdul['SHUTTER_INFO'].data)

Copy the rate files for the single exposure we've selected (corresponding to the first nod position) so that the pipeline products don't get overwritten. Also, we need to change the metafile header keyword to use the new file we created in the previous cell.

In [ ]:
if (runflag == True):
    copyfile(output_dir+'nirspec_mossim_d1_g235m_nrs1_rate.fits',
             output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_rate.fits')
    hdul = fits.open(output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_rate.fits', mode='update')
    print(hdul[0].header['MSAMETFL'])
    hdul[0].header['MSAMETFL'] = 'newmetafile_1shutter_msa.fits'
    print(hdul[0].header['MSAMETFL'])
    hdul.close()

    copyfile(output_dir+'nirspec_mossim_d1_g235m_nrs2_rate.fits',
             output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_rate.fits')
    hdul = fits.open(output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_rate.fits', mode='update')
    print(hdul[0].header['MSAMETFL'])
    hdul[0].header['MSAMETFL'] = 'newmetafile_1shutter_msa.fits'
    print(hdul[0].header['MSAMETFL'])
    hdul.close()

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# reprocess through spec2

if (runflag == True):
    input_file = output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_rate.fits'
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    spec2.master_background.save_background = True  # output the master background spectrum
    result = spec2(input_file)
    
    input_file = output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_rate.fits'
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    spec2.master_background.save_background = True  # output the master background spectrum
    result = spec2(input_file)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

Next, we will repeat the above, but now adding in one-shutter slitlets marked as being open to background. The pipeline will read this metafile and automatically identify that a master background needs to be constructed and subtracted from each slitlet. This simulation was not specifically designed with dedicated background slitlets, so we will create them by hand using background shutters in slitlets where the source flux was not detectable.

In [ ]:
if (runflag == True):

    hdul = fits.open(metafile)
    
    # pick the same source shutter as before
    # add shutters from other slitlets that are not contaminated by source flux to use as dedicated background slitlets
    indices = [255, 261, 264, 267, 288, 291, 294, 324, 327, 330, 360, 363, 366]

    slitlets = hdul['SHUTTER_INFO'].data['SLITLET_ID'][indices]
    metaids = hdul['SHUTTER_INFO'].data['MSA_METADATA_ID'][indices]
    quads = hdul['SHUTTER_INFO'].data['SHUTTER_QUADRANT'][indices]
    rows = hdul['SHUTTER_INFO'].data['SHUTTER_ROW'][indices]
    cols = hdul['SHUTTER_INFO'].data['SHUTTER_COLUMN'][indices]
    sources = hdul['SHUTTER_INFO'].data['SOURCE_ID'][indices]
    bkgd = hdul['SHUTTER_INFO'].data['BACKGROUND'][indices]
    state = hdul['SHUTTER_INFO'].data['SHUTTER_STATE'][indices]
    srcx = hdul['SHUTTER_INFO'].data['ESTIMATED_SOURCE_IN_SHUTTER_X'][indices]
    srcy = hdul['SHUTTER_INFO'].data['ESTIMATED_SOURCE_IN_SHUTTER_Y'][indices]
    dith = hdul['SHUTTER_INFO'].data['DITHER_POINT_INDEX'][indices]
    primary = hdul['SHUTTER_INFO'].data['PRIMARY_SOURCE'][indices]

    # set BACKGROUND=Y and SOURCE=N for all shutters we want to use to construct the master background
    bkgd[1:12] = 'Y'
    primary[1:12] = 'N'
    # give these different slitlet numbers so the pipeline will extract them separately
    slitlets = [31, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111]

    print(slitlets)
    print(bkgd)
    print(primary)
    
    # construct a new table with just these elements
    tabcol1 = fits.Column(name='SLITLET_ID', format='I', array=slitlets)
    tabcol2 = fits.Column(name='MSA_METADATA_ID', format='I', array=metaids)
    tabcol3 = fits.Column(name='SHUTTER_QUADRANT', format='I', array=quads)
    tabcol4 = fits.Column(name='SHUTTER_ROW', format='I', array=rows)
    tabcol5 = fits.Column(name='SHUTTER_COLUMN', format='I', array=cols)
    tabcol6 = fits.Column(name='SOURCE_ID', format='I', array=sources)
    tabcol7 = fits.Column(name='BACKGROUND', format='A', array=bkgd)
    tabcol8 = fits.Column(name='SHUTTER_STATE', format='4A', array=state)
    tabcol9 = fits.Column(name='ESTIMATED_SOURCE_IN_SHUTTER_X', format='E', array=srcx)
    tabcol10 = fits.Column(name='ESTIMATED_SOURCE_IN_SHUTTER_Y', format='E', array=srcy)
    tabcol11 = fits.Column(name='DITHER_POINT_INDEX', format='I', array=dith)
    tabcol12 = fits.Column(name='PRIMARY_SOURCE', format='1A', array=primary)
    hdul['SHUTTER_INFO'] = fits.BinTableHDU.from_columns([tabcol1, tabcol2, tabcol3, tabcol4, tabcol5,
                                                          tabcol6, tabcol7, tabcol8, tabcol9, tabcol10,
                                                          tabcol11, tabcol12], name='SHUTTER_INFO')

    # save to a new metafile
    hdul.writeto(output_dir+'newmetafile_1shutter+background_msa.fits')
    hdul.close()

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# take a look at the contents of this new metafile

metafile = output_dir+'newmetafile_1shutter+background_msa.fits'
hdul = fits.open(metafile)

print(hdul['SHUTTER_INFO'].data)

Once again, copy the rate files for the single exposure we've selected (corresponding to the first nod position) so that the pipeline products don't get overwritten. Also, we need to change the metafile header keyword to use the new file we created in the previous cell.

In [ ]:
if (runflag == True):
    
    copyfile(output_dir+'nirspec_mossim_d1_g235m_nrs1_rate.fits',
             output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_MBS_rate.fits')
    hdul = fits.open(output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_MBS_rate.fits', mode='update')
    print(hdul[0].header['MSAMETFL'])
    hdul[0].header['MSAMETFL'] = 'newmetafile_1shutter+background_msa.fits'
    print(hdul[0].header['MSAMETFL'])
    hdul.close()

    copyfile(output_dir+'nirspec_mossim_d1_g235m_nrs2_rate.fits',
             output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_MBS_rate.fits')
    hdul = fits.open(output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_MBS_rate.fits', mode='update')
    print(hdul[0].header['MSAMETFL'])
    hdul[0].header['MSAMETFL'] = 'newmetafile_1shutter+background_msa.fits'
    print(hdul[0].header['MSAMETFL'])
    hdul.close()

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

Now we will rerun the calwebb_spec2 pipeline using the new metafile. The processing proceeds as follows:

* All background slitlets are processed first, calibrated as an extended source. If there are multiple background slitlets, all the extracted 1D spectra are averaged together to create a single master background spectrum.
* Each source slitlet is processed through part of the spec2 pipeline up to the master_background step (including the assign_wcs, imprint, msaflagopen, extract_2d, and srctype steps).
* The 1D master background is then resampled into the unrectified 2D space of the source slitlet, and the various throughput-related corrections (e.g., flat_field, barshadow, etc) are *removed*.
* The 2D background is subtracted from the 2D source spectrum.
* The remaining spec2 steps are run on the subtracted source spectrum.

In [ ]:
if (runflag == True):
    input_file = output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_MBS_rate.fits'
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    spec2.master_background.save_background = True  # output the master background spectrum
    result = spec2(input_file)
    
    input_file = output_dir+'nirspec_mossim_d1_g235m_nrs2_1shutter_MBS_rate.fits'
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    spec2.master_background.save_background = True  # output the master background spectrum
    result = spec2(input_file)

ValueError: invalid literal for int() with base 10: " E712 comparison to True should be 'if cond is True"

In [ ]:
# show the results

calfile = output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_MBS_cal.fits'
cal = datamodels.open(calfile)
root = calfile[:-9]
s2d = datamodels.open(root+'_s2d.fits')  # this contains the calibrated *rectified* 2D spectrum
x1d = datamodels.open(root+'_x1d.fits')  # this contains the aperture-extracted 1D spectrum
mb2d = datamodels.open(root+'_masterbg2d.fits')  # this contains the 2D master background spectrum
mbx1d = datamodels.open(root+'_masterbg1d.fits')  # this contains the 1D master background spectrum

# first, display the master background spectrum that was created
# the combined 1D spectrum
mbx1dwave = mbx1d.spec[0].spec_table.WAVELENGTH
mbx1dflux = mbx1d.spec[0].spec_table.FLUX
fig = plt.figure(figsize=(19, 8))
plt.plot(mbx1dwave, mbx1dflux)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.title('1D master background')
plt.ylim(-2.e3, 1.5e4)
plt.show()

# 2D MB (resampled into the unrectified 2D space of slitlet 31)
mb2dsci = mb2d.slits[0].data
show_image(mb2dsci, 0.0, 0.02, aspect=10., scale='linear', units='MJy')

# now show the source spectrum
for i, slit in enumerate(cal.slits):
    # let's look at the subtracted source
    if (slit.name == '31'):
        print(slit.name)
        
        calsci = slit.data  # contains the pixel data from the cal file (SCI extension)
        s2dsci = s2d.slits[i].data  # contains the pixel data from the s2d file
    
        # determine the wavelength scale of the s2d data for plotting purposes
        # get the data model WCS object
        wcsobj = s2d.slits[i].meta.wcs
        y, x = np.mgrid[:s2dsci.shape[0], : s2dsci.shape[1]]  # grid of pixel x,y indices
        det2sky = wcsobj.get_transform('detector','world')  # the coordinate transform from detector space (pixels) to sky (RA, DEC in degrees)
        ra, dec, s2dwave = det2sky(x, y)  # RA, Dec, wavelength (microns) for each pixel
        s2dwaves = s2dwave[0, :]  # only need a single row of values since this is the rectified spectrum
        xtint = np.arange(100, s2dsci.shape[1], 100)
        xtlab = np.round(s2dwaves[xtint], 2)  # wavelength labels for the x-axis
        
        # get wavelength & flux from the x1d data model
        x1dwave = x1d.spec[i].spec_table.WAVELENGTH
        x1dflux = x1d.spec[i].spec_table.FLUX
  
        # plot the unrectified calibrated 2D spectrum
        show_image(calsci, -0.01, 0.01, aspect=10., scale='linear', units='MJy')
        
        # plot the rectified 2D spectrum
        show_image(s2dsci, -0.01, 0.01, aspect=10., scale='linear', units='MJy')
        plt.xticks(xtint, xtlab)
        plt.xlabel('wavelength (microns)')
        
        # plot the 1D extracted spectrum
        fig = plt.figure(figsize=(19, 8))
        plt.plot(x1dwave, x1dflux)
        plt.xlabel('wavelength (microns)')
        plt.ylabel('flux (Jy)')
        plt.title('background-subtracted source')
        plt.ylim(-1.e4, 1.e6)
        plt.show()

2021-10-18 17:00:51,662 - stpipe - INFO - 40:50: E231 missing whitespace after ','


ValueError: Column names don't match schema. Schema has {'berror', 'error'}. Data has {'sb_var_flat', 'flux_var_rnoise', 'flux_var_poisson', 'flux_var_flat', 'bkgd_var_poisson', 'flux_error', 'bkgd_var_rnoise', 'bkgd_error', 'bkgd_var_flat', 'sb_var_poisson', 'sb_var_rnoise'}

Finally, let's compare all three x1d products generated for this source: the unsubtracted version, the nod-subtracted product, and the master background-subtracted product.

In [ ]:
x1dnosub = datamodels.open(output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_x1d.fits')
x1dnodsub = datamodels.open(output_dir+'nirspec_mossim_d1_g235m_nrs1_x1d.fits')
x1dmbsub = datamodels.open(output_dir+'nirspec_mossim_d1_g235m_nrs1_1shutter_MBS_x1d.fits')

x1dnosubwave = x1dnosub.spec[0].spec_table.WAVELENGTH
x1dnosubflux = x1dnosub.spec[0].spec_table.FLUX

# need to loop through all the spectra for the nod-subtracted case to find source 2140
for spec in x1dnodsub.spec:
    if (spec.source_id == 2140):
        x1dnodsubwave = spec.spec_table.WAVELENGTH
        x1dnodsubflux = spec.spec_table.FLUX

x1dmbsubwave = x1dmbsub.spec[12].spec_table.WAVELENGTH
x1dmbsubflux = x1dmbsub.spec[12].spec_table.FLUX
    
# plot the 1D spectra
fig = plt.figure(figsize=(19,8))
plt.plot(x1dnosubwave, x1dnosubflux)
plt.plot(x1dnodsubwave, x1dnodsubflux)
plt.title('results of nod subtraction')
plt.ylim(0.,6e4)
#plt.xlim(1.65,2.0)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.show()

# plot the 1D extracted spectrum
fig = plt.figure(figsize=(19,8))
plt.plot(x1dnosubwave, x1dnosubflux)
plt.plot(x1dmbsubwave, x1dmbsubflux)
plt.title('results of master background subtraction')
plt.ylim(0.,6e4)
#plt.xlim(1.65,2.0)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.show()

2021-10-18 17:00:57,152 - stpipe - INFO - 18:29: E231 missing whitespace after ','
2021-10-18 17:00:57,154 - stpipe - INFO - 22:12: E231 missing whitespace after ','
2021-10-18 17:00:57,154 - stpipe - INFO - 23:1: E265 block comment should start with '# '
2021-10-18 17:00:57,155 - stpipe - INFO - 29:29: E231 missing whitespace after ','
2021-10-18 17:00:57,156 - stpipe - INFO - 33:12: E231 missing whitespace after ','
2021-10-18 17:00:57,157 - stpipe - INFO - 34:1: E265 block comment should start with '# '


ValueError: Column names don't match schema. Schema has {'berror', 'error'}. Data has {'sb_var_flat', 'flux_var_rnoise', 'flux_var_poisson', 'flux_var_flat', 'bkgd_var_poisson', 'flux_error', 'bkgd_var_rnoise', 'bkgd_error', 'bkgd_var_flat', 'sb_var_poisson', 'sb_var_rnoise'}